# Engines to test:
### 1. Tesseract
### 2. PaddleOCR
### 3. EasyOCR

## Dataset:

https://guillaumejaume.github.io/FUNSD/

In [ ]:
import os
import json
import pandas as pd
import cv2
import matplotlib.pyplot as plt

annotations_folder = ''
images_folder = ''

data = []

for filename in os.listdir(annotations_folder):
    if filename.endswith('.json'):
        with open(os.path.join(annotations_folder, filename), 'r') as file:
            annotation_data = json.load(file)
            image_filename = filename.split('.')[0] + '.png'
            text = ""
            for item in annotation_data['form']:
                text += item['text'] + " "

            data.append({'filename': image_filename, 'text': text.strip()})

df = pd.DataFrame(data)

df.head()

: 